### Word to Vec

In [1]:
#! pip install plotly_express

In [1]:
import gensim

/Users/rehanmerchant/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
BAG = OHCO[:4] # Paragraphs
# BAG = OHCO[:5] # Sentences
window = 5

In [3]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px

In [4]:
%matplotlib inline

In [5]:
LIB = pd.read_csv('LIB.csv')

In [6]:
for i in LIB['book_title']:
    if 'Austen' in i:
        print(i)

Mansfield Park, by Jane Austen
Northanger Abbey, by Jane Austen
Pride and Prejudice, by Jane Austen, Edited


In [7]:
aus_books = []
for i in LIB.iterrows():
    if 'austen' in i[1][3]:
        aus_books.append(i[1][0])

    

In [8]:
aus_books

[141, 121, 42671]

In [10]:
TOKENS = pd.read_csv('TOKEN2.csv')

In [11]:
t1=TOKENS

In [12]:
corpus1 = t1[~t1.pos.str.match('NNPS?')]\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()


In [13]:
m1 = word2vec.Word2Vec(corpus1, window=window, min_count=200, workers=4)

In [14]:
# coords = pd.DataFrame(index=range(len(model.wv)))
# coords['label'] = [w for w in model.wv]
# coords['vector'] = coords['label'].apply(lambda x: model.wv.get_vector(x))
coords1 = pd.DataFrame(index=range(len(m1.wv.key_to_index)))
coords1['label'] = m1.wv.index_to_key
coords1['vector'] = coords1['label'].apply(lambda x: m1.wv.get_vector(x))

In [15]:
tsne_model1 = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_values1 = tsne_model1.fit_transform(coords1['vector'].tolist())

In [16]:
coords1['x'] = tsne_values1[:,0]
coords1['y'] = tsne_values1[:,1]
coords1.head()

,label,vector,x,y
0,the,"[-0.16552766, 0.56497824, 0.50821096, 0.178076...",4.114814,6.481851
1,to,"[-0.17420313, 0.35459864, -0.06438009, 0.20676...",-1.406842,4.051281
2,and,"[-0.1162485, 0.2775262, 0.15647973, 0.17467678...",4.827510,1.866202
3,of,"[-0.31989172, 0.4158327, -0.09709886, 0.134807...",6.642979,4.514800
4,her,"[-0.5099739, 0.52677894, 0.14838536, 0.0034291...",8.767496,2.407175


In [17]:
px.scatter(coords1, 'x', 'y', text='label', height=1000, title='Jane Austen Word Plot').update_traces(mode='text')

In [18]:
def complete_analogy_1(A, B, C, n=2):
    try:
        return m1.wv.most_similar(positive=[B, C], negative=[A])[0:n]
    except KeyError as e:
        print('Error:', e)
        return None

In [19]:
complete_analogy_1('mother', 'herself', 'father')

[('even', 0.8950825929641724), ('though', 0.8659785985946655)]

In [20]:
complete_analogy_1('sister', 'me', 'brother')

[('us', 0.8789187669754028), ('you', 0.8471131324768066)]

In [21]:
complete_analogy_1('you', 'thought', 'me')

[('felt', 0.7536299228668213), ('speak', 0.7002220749855042)]